## Part 1 - Extraction and Frequencies

In [23]:
from nltk.corpus.reader.bnc import BNCCorpusReader
from nltk.collocations import *
from collections import Counter
from sklearn.model_selection import train_test_split
from nltk.lm import MLE
from nltk.util import ngrams
from nltk.corpus import stopwords
import nltk, re, pprint, string
from nltk import word_tokenize, sent_tokenize
import numpy
import random
import mpmath as mp
import sys

In [6]:
bnc_reader = BNCCorpusReader(root="BNC/Texts", fileids=r'[A-K]/\w*/\w*\.xml')
fileids = ['aca/A6U.xml']

In [7]:
raw_sents = BNCCorpusReader.sents(bnc_reader, fileids=fileids)
punct = "“”‘’!\"#$€%&()*'+-,./:;<=>?@[\]^_`{|}~\n"
temp = []
        
for sentence in raw_sents:
    temp.append("<s>")
    for word in sentence:
        if word not in punct:
            temp.append(word)
    temp.append("</s>")
    
tokens = [x.lower() for x in temp]
# word_list = []
# for word in tokens:
#     if word is not word_list:
#         word_list.append(word)

In [8]:
frequencies = []
for instance in tokens:
    frequencies.append(tokens.count(instance))
    
word_freq = list(zip(tokens, frequencies))
print(word_freq)

[('<s>', 1082), ('being', 16), ('drawn', 6), ('to', 617), ('an', 163), ('image', 24), ('</s>', 1082), ('<s>', 1082), ('guy', 3), ('brett', 1), ('</s>', 1082), ('<s>', 1082), ('why', 8), ('do', 14), ('certain', 13), ('images', 22), ('matter', 11), ('to', 617), ('one', 82), ('and', 780), ('why', 8), ('is', 485), ('the', 2109), ('desire', 6), ('to', 617), ('answer', 6), ('this', 184), ('question', 10), ('as', 288), ('involuntary', 2), ('as', 288), ('the', 2109), ('response', 12), ('itself', 23), ('</s>', 1082), ('<s>', 1082), ('why', 8), ('does', 25), ('it', 215), ('seem', 8), ('important', 17), ('that', 247), ('the', 2109), ('answer', 6), ('should', 11), ('have', 85), ('some', 40), ('objective', 2), ('quality', 3), ('about', 32), ('it', 215), ('an', 163), ('insight', 1), ('into', 42), ('history', 32), ('society', 12), ('knowledge', 8), ('rather', 23), ('than', 44), ('point', 15), ('to', 617), ('a', 619), ('merely', 12), ('personal', 6), ('obsession', 3), ('</s>', 1082), ('<s>', 1082), ('

In [9]:
train_words, test_words = train_test_split(tokens, test_size = 0.2)

## Part 2 - Language Models

### Vanilla Model

In [10]:
def vanilla_uni(train_words):
    unigram = Counter(train_words)
    
    for word in unigram:
        unigram[word] = unigram[word]/len(train_words)
        
    return unigram
def vanilla_bi(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    counter = Counter(train_words)
    
    for word in bigram:
        bigram[word] = bigram[word]/counter[word[0]]
        
    return bigram
    
def vanilla_tri(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    trigram = Counter([(word, train_words[i + 1], train_words[i + 2]) for i, word in enumerate(train_words[:-2])])
    
    for word in trigram:
        trigram[word] = trigram[word]/bigram[(word[0], word[1])]
        
    return trigram

### Laplace Model

In [11]:
def laplace_uni(train_words):
    unigram = Counter(train_words)
    
    for word in unigram:
        unigram[word] = unigram[word]+1/len(train_words)
        
    return unigram
def laplace_bi(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    counter = Counter(train_words)
    
    for word in bigram:
        bigram[word] = bigram[word]+1/counter[word[0]]
        
    return bigram
    
def laplace_tri(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    trigram = Counter([(word, train_words[i + 1], train_words[i + 2]) for i, word in enumerate(train_words[:-2])])
    
    for word in trigram:
        trigram[word] = trigram[word]+1/bigram[(word[0], word[1])]
        
    return trigram

### UNK Model

In [12]:
def unk_uni(train_words):
    
    counter = Counter(train_words)
    model = {}
    model["<UNK>"] = 0
    
    for word in counter:
        if counter[word] <= 2:
            model["<UNK>"] += 1
            
        else:
            model[word] = counter[word]
        
    return laplace_uni(train_words)

def unk_bi(train_words):
    
    unigram_model = unk_uni(train_words)
    
    for i, word in enumerate(train_words):
        if not (word in unigram_model):
            train_words[i] = "<UNK>"
            
    return laplace_bi(train_words)

def unk_tri(train_words):
    
    unigram = unk_uni(train_words)
    
    for i, word in enumerate(train_words):
        if not (word in unigram):
            train_words[i] = "<UNK>"
            
    return laplace_tri(train_words)

### Probability

In [13]:
def uni_prob(model,unigram):
    total = sum(model.values())
    top = model[unigram]
    if top == 0:
        return 0
    probability = top/total
    return probability

def bi_prob(model_bi, model_uni, bigram):
    first = bigram.split()[0]
    second = bigram.split()[1]
    total = model_uni[first]
    top = model_bi[first,second]
    if top == 0:
        return 0
    if total == 0:
        return 0    
    probability = top/total
    return probability

def tri_prob(model_tri, model_bi, trigram):
    first = trigram.split()[0]
    second = trigram.split()[1]
    third = trigram.split()[2]
    total = model_bi[second,third]
    top = model_tri[first,second,third]
    if top == 0:
        return 0
    if total == 0:
        return 0  
    probability = top/total
    return probability

In [14]:
def probability (sentence, model):
    sent = "<s> "+ sentence + " </s>"
    words = sent.split()
    uni_lambda = 0.1
    bi_lambda = 0.3
    tri_lambda = 0.6
    
    unigrams_probability = []
    bigrams_probability = []
    trigrams_probability = []
    
    if model == "Vanilla":
        # unigram
                
        for word in words:
            unigrams_probability.append(uni_prob(vanilla_uni(train_words),word))
        
        # bigram
        
        bigrams = nltk.ngrams(words, 2)
        for pair in bigrams:
            bigram = ' '.join(pair)
            bigrams_probability.append(bi_prob(vanilla_bi(train_words), vanilla_uni(train_words), bigram))
        
        # trigram
        trigrams = nltk.ngrams(words, 3)
        for trio in trigrams:
            trigram = ' '.join(trio)
            trigrams_probability.append(tri_prob(vanilla_tri(train_words),vanilla_bi(train_words),trigram))
        
    elif model == "Laplace":
        # unigram
                
        for word in words:
            unigrams_probability.append(uni_prob(laplace_uni(train_words),word))
        
        # bigram
        
        bigrams = nltk.ngrams(words, 2)
        for pair in bigrams:
            bigram = ' '.join(pair)
            bigrams_probability.append(bi_prob(laplace_bi(train_words), laplace_uni(train_words), bigram))
        
        # trigram
        trigrams = nltk.ngrams(words, 3)
        for trio in trigrams:
            trigram = ' '.join(trio)
            trigrams_probability.append(tri_prob(laplace_tri(train_words),laplace_bi(train_words),trigram))
        
    elif model == "UNK":
        # unigram
                
        for word in words:
            unigrams_probability.append(uni_prob(unk_uni(train_words),word))
        
        
        # bigram
        
        bigrams = nltk.ngrams(words, 2)
        for pair in bigrams:
            bigram = ' '.join(pair)
            bigrams_probability.append(bi_prob(unk_bi(train_words), unk_uni(train_words), bigram))
        
        # trigram
        trigrams = nltk.ngrams(words, 3)
        for trio in trigrams:
            trigram = ' '.join(trio)
            trigrams_probability.append(tri_prob(unk_tri(train_words),unk_bi(train_words),trigram))
        
    prob1 = numpy.prod(unigrams_probability)
    prob2 = numpy.prod(bigrams_probability)
    prob3 = numpy.prod(trigrams_probability) 
    
    probability = (uni_lambda*prob1)+(bi_lambda*prob2)+(tri_lambda*prob3)
    return probability

In [15]:
print (probability ("why does it seem important that the answer",model="UNK"))

4.215441149135651e-27


### Perplexity

In [24]:
def perplexity(test_words, model):
    
    perp = mp.mpf(1)
    
    N = mp.mpf(0)
    
    for line in test_words:
        N += len(line)
        line = ' '.join(line)
        
        if model[line] > 0:
            perp = perp * (1/model[line])
        else:
            perp = perp * sys.maxsize
            
    perp = pow(perp, 1/float(N))
    return perp

In [25]:
perplexity(test_words, vanilla_bi(train_words))

mpf('6894.2962883617947')

# Generation

In [87]:
def generate(model,phrase):
    model = vanilla_bi(train_words)
    new_sentence = ["<s>"]
    text = phrase.split()
    end = False
    for word in text:
        new_sentence.append(word)
    last = text[-1]
    best_prob = 0
    
    bigrams = []
    
    for entry in model:
        if entry[0] == last:
            bigrams.append(entry)
            
    for pair in bigrams:
        text = ' '.join(pair)
        prob = probability(text, model = "Vanilla")
        if prob > best_prob:
            best_prob = prob
            best_word = pair[1]
            
    new_sentence.append(best_word)
    if best_word == "</s>":
        end = True
            



In [88]:
model = "2"
phrase = "of the"
generate(model,phrase)

references 1.658168488005196e-09
first 5.306139161616628e-09
which 0.24321968823634887
modernism 0.3441475198371671
the 0.5155041464514781
inverted 9.485565824884725
structure 16.86322813176502
trends 167.4734010033496


KeyboardInterrupt: 

In [51]:
model = input("Which model would you like to use? (1) Vanilla (2) Laplace (3) UNK : ")
phrase = input("Enter phrase to continue to generate: ")
generate(model,phrase)

Which model would you like to use? (1) Vanilla (2) Laplace (3) UNK : 1
Enter phrase to continue to generate: testing the water
